In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install tensorflow
!pip install openpyxl
!pip install scikit-learn==1.5.2 scikeras==0.13.0

In [ ]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from sklearn.neural_network import MLPRegressor

import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

# Parametreler
start_date = "2018-01-01"
end_date = "2018-12-31"

# Tahmin periyotları - 3 farklı senaryo
TAHMIN_PERIYOTLARI = {
 '1_gun': 1,    # 03 Aralık 2018 (1 iş günü)
 '10_gun': 10,  # 03-14 Aralık 2018 (10 iş günü = 2 hafta)
 '21_gun': 21   # 03-31 Aralık 2018 (21 iş günü = 1 ay)
}

# Şu an için varsayılan değer (döngüde değişecek)
# Aktif periyot seçimi - değiştirmek için '1_gun', '10_gun' veya '21_gun' yazın
AKTIF_PERIYOT = '21_gun'
tahminEdilecekGunSayisi = TAHMIN_PERIYOTLARI[AKTIF_PERIYOT]
time_step = 30 # time_step >= tahminEdilecekGunSayisi olmalı
epoch_sayisi = 100

# CSV dosyası yolu
DATA_FILE = "/content/drive/MyDrive/Colab Notebooks/tez/Data/BIST_50_2018_Data.csv"

plt.style.use("fivethirtyeight")

In [ ]:
# CSV'den veri okuma
print(f" Veri yükleniyor: {DATA_FILE}")

df_raw = pd.read_csv(DATA_FILE)

# Veri temizleme
df = df_raw.copy()

# Sütun adını düzelt (Now -> Close)
df = df.rename(columns={'Now': 'Close'})

# Tarih formatını düzelt (31.12.2018 -> 2018-12-31)
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')

# Sayı formatlarını düzelt (Türkçe format: virgül -> nokta)
numeric_cols = ['Close', 'Open', 'High', 'Low']
for col in numeric_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace('.', '', regex=False) # Binlik ayracı kaldır
        df[col] = df[col].str.replace(',', '.', regex=False) # Virgülü noktaya çevir
        df[col] = df[col].astype(float)

# Sıralamayı düzelt (eskiden yeniye)
df = df.sort_values('Date').reset_index(drop=True)

# Index olarak tarih ata
df = df.set_index('Date')

# Tarih filtreleme
df = df[(df.index >= start_date) & (df.index <= end_date)]

print(f" Toplam veri: {len(df)} iş günü")
print(f" Tarih aralığı: {df.index[0].strftime('%Y-%m-%d')} → {df.index[-1].strftime('%Y-%m-%d')}")
df.head()

In [ ]:
# Sadece Kapanış değerlerinin olduğu bir dataframe oluştur
df1 = df[['Close']]
df1.head()

In [ ]:
# Grafiği çizme
plt.figure(figsize=(15, 8))
plt.plot(df1, label="Kapanış")
plt.title("BIST50 Kapanış Fiyatları")
plt.xlabel("Tarih")
plt.ylabel("Kapanış Fiyatı")
plt.show()

In [ ]:
# Grafiği çizme (İngilizce)
plt.figure(figsize=(15, 8))
plt.plot(df1, label="Close")
plt.title("BIST50 Closing Prices")
plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.show()

In [ ]:
# Veriyi ayırma
train = df1[:-(time_step+2)]
test = df1[-(time_step+2):]

# Veriyi ölçekleme
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
train.tail()

In [ ]:
test

In [ ]:
# Dataset oluşturma
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        dataX.append(dataset[i:(i+time_step), 0])
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

X_train, y_train = create_dataset(scaled_train, time_step)
X_test, y_test = create_dataset(scaled_test, time_step)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
# Veriyi yeniden şekillendirme
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
#Cross Validation
!pip install scikeras
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Keras model oluşturma fonksiyonları
def build_lstm_model(input_shape):
 model = Sequential([
 LSTM(50, return_sequences=True, input_shape=input_shape),
 LSTM(50, return_sequences=True),
 LSTM(50),
 Dense(1)
 ])
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model

def build_rnn_model(input_shape):
 model = Sequential([
 SimpleRNN(50, return_sequences=True, input_shape=input_shape),
 SimpleRNN(50, return_sequences=True),
 SimpleRNN(50),
 Dense(1)
 ])
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model

def build_cnn_model(input_shape):
 model = Sequential([
 Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
 MaxPooling1D(pool_size=2),
 Flatten(),
 Dense(50, activation='relu'),
 Dense(1)
 ])
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model

def build_mlp_model(input_shape):
 model = Sequential([
 Dense(50, activation='relu', input_dim=input_shape[0]),
 Dense(1)
 ])
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model

# Cross-validation için RMSE scorer'ını tanımla
def rmse(y_true, y_pred):
 return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Modeli ve cross-validation'ı ayarla
input_shape = (X_train.shape[1], X_train.shape[2]) # Örneğin: (time_step, feature)
models = {
 'LSTM': KerasRegressor(model=build_lstm_model, model__input_shape=input_shape),
 'RNN': KerasRegressor(model=build_rnn_model, model__input_shape=input_shape),
 'CNN': KerasRegressor(model=build_cnn_model, model__input_shape=input_shape),
 'MLP': KerasRegressor(model=build_mlp_model, model__input_shape=(X_train.shape[1],))
}

# Her model için cross-validation uygula
results = {}
for name, model in models.items():
 cv = KFold(n_splits=10, shuffle=True, random_state=42)
 cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=rmse_scorer)
 results[name] = cv_scores
 print(f"{name}: CV RMSE mean={-cv_scores.mean()}, std={cv_scores.std()}")


In [ ]:
# Modelleri oluştur ve eğit
# LSTM Modeli
model_lstm = Sequential([
 LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
 LSTM(50, return_sequences=True),
 LSTM(50),
 Dense(1)
])
model_lstm.compile(loss='mean_squared_error', optimizer='adam')
model_lstm.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=64, verbose=1)

# RNN Modeli
model_rnn = Sequential([
 SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
 SimpleRNN(50, return_sequences=True),
 SimpleRNN(50),
 Dense(1)
])
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
model_rnn.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=64, verbose=1)


# CNN Modeli
model_cnn = Sequential([
 Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)), # kernel_size = 2
 MaxPooling1D(pool_size=2),
 Flatten(),
 Dense(50, activation='relu'),
 Dense(1)
])

model_cnn.compile(loss='mean_squared_error', optimizer='adam')
model_cnn.fit(X_train, y_train, epochs=epoch_sayisi, batch_size=64, verbose=1)


# MLP Modeli
model_mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=epoch_sayisi)
model_mlp.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]), y_train)

In [ ]:
df1

In [ ]:
scaled_df1 = scaler.transform(df1)

def make_predictions(model, is_mlp=False):
    X, y = create_dataset(scaled_df1, time_step)
    if is_mlp:
        X_predict = model.predict(X[-tahminEdilecekGunSayisi:, :])
    else:
        X_3D = X.reshape(X.shape[0], X.shape[1], 1)
        X_predict = model.predict(X_3D[-tahminEdilecekGunSayisi:, :])
    return scaler.inverse_transform(X_predict.reshape(-1, 1))

# Tahminleri al
lstm_predictions = make_predictions(model_lstm)
rnn_predictions = make_predictions(model_rnn)
cnn_predictions = make_predictions(model_cnn)
mlp_predictions = make_predictions(model_mlp, is_mlp=True)


# Performans ölçütleri
def calculate_metrics(true_values, predictions):
    rmse = math.sqrt(mean_squared_error(true_values, predictions))
    relative_rmse = rmse / np.mean(true_values)
    mape = np.mean(np.abs((true_values - predictions) / true_values)) * 100
    return rmse, relative_rmse, mape

In [ ]:
lstm_predictions

In [ ]:
# Tahminler için doğru tarih aralığını al
tahmin_tarihleri = df.index[-tahminEdilecekGunSayisi:]

# Grafik çizme
plt.figure(figsize=(15, 8))
plt.title('Kapanış Fiyatları ve Tahminler')
plt.xlabel('Tarih', fontsize=18)
plt.ylabel('Kapanış Fiyatları', fontsize=18)

#plt.plot(train.index, train['Close'], label='Train', color='blue')
#plt.plot(test.index, test['Close'], label='Test', color='green')

#plt.plot(df.index[:-(tahminEdilecekGunSayisi-1)], df[:-(tahminEdilecekGunSayisi-1)]['Close'], label='Train (Gerçek)', color='blue')
#plt.plot(df.index[-tahminEdilecekGunSayisi:], df[-tahminEdilecekGunSayisi:]['Close'], label='Test (Gerçek)', color='green')
plt.plot(df.index[:], df[:]['Close'], label='Gerçek Değerler', color='blue')


plt.plot(tahmin_tarihleri, lstm_predictions, label='LSTM Tahminleri',color='green')
plt.plot(tahmin_tarihleri, mlp_predictions, label='MLP Tahminleri', color='orange')
#plt.plot(tahmin_tarihleri, rnn_predictions, label='RNN Tahminleri')
#plt.plot(tahmin_tarihleri, cnn_predictions, label='CNN Tahminleri')

plt.legend(loc='lower left')
plt.show()

In [ ]:
# Tahminler için doğru tarih aralığını al (İngilizce)
tahmin_tarihleri = df.index[-tahminEdilecekGunSayisi:]

# Grafik çizme
plt.figure(figsize=(15, 8))
plt.title('Closing Prices and Predictions')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Closing Prices', fontsize=18)

#plt.plot(train.index, train['Close'], label='Train', color='blue')
#plt.plot(test.index, test['Close'], label='Test', color='green')

#plt.plot(df.index[:-(tahminEdilecekGunSayisi-1)], df[:-(tahminEdilecekGunSayisi-1)]['Close'], label='Train (Gerçek)', color='blue')
#plt.plot(df.index[-tahminEdilecekGunSayisi:], df[-tahminEdilecekGunSayisi:]['Close'], label='Test (Gerçek)', color='green')
plt.plot(df.index[:], df[:]['Close'], label='Actual Values', color='blue')


plt.plot(tahmin_tarihleri, lstm_predictions, label='LSTM Predictions',color='green')
plt.plot(tahmin_tarihleri, mlp_predictions, label='MLP Predictions', color='orange')
#plt.plot(tahmin_tarihleri, rnn_predictions, label='RNN Tahminleri')
#plt.plot(tahmin_tarihleri, cnn_predictions, label='CNN Tahminleri')

plt.legend(loc='lower left')
plt.show()

In [ ]:
# Tahminler için doğru tarih aralığını al
tahmin_tarihleri = df.index[-tahminEdilecekGunSayisi:]

# Grafik çizme
plt.figure(figsize=(15, 8))
plt.title('Kapanış Fiyatları ve Tahminler')
plt.xlabel('Tarih', fontsize=18)
plt.ylabel('Kapanış Fiyatları', fontsize=18)

#plt.plot(train.index, train['Close'], label='Train', color='blue')
#plt.plot(test.index, test['Close'], label='Test', color='green')

#plt.plot(df.index[:-(tahminEdilecekGunSayisi-1)], df[:-(tahminEdilecekGunSayisi-1)]['Close'], label='Train (Gerçek)', color='blue')
#plt.plot(df.index[-tahminEdilecekGunSayisi:], df[-tahminEdilecekGunSayisi:]['Close'], label='Test (Gerçek)', color='green')
plt.plot(df.index[:], df[:]['Close'], label='Gerçek Değerler', color='blue')


#plt.plot(tahmin_tarihleri, lstm_predictions, label='LSTM Tahminleri')
#plt.plot(tahmin_tarihleri, mlp_predictions, label='MLP Tahminleri')
plt.plot(tahmin_tarihleri, rnn_predictions, label='RNN Tahminleri', color='purple')
plt.plot(tahmin_tarihleri, cnn_predictions, label='CNN Tahminleri', color='red')

plt.legend(loc='lower left')
plt.show()

In [ ]:
# Tahminler için doğru tarih aralığını al (İngilizce)
tahmin_tarihleri = df.index[-tahminEdilecekGunSayisi:]

# Grafik çizme
plt.figure(figsize=(15, 8))
plt.title('Closing Prices and Predictions')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Closing Prices', fontsize=18)

#plt.plot(train.index, train['Close'], label='Train', color='blue')
#plt.plot(test.index, test['Close'], label='Test', color='green')

#plt.plot(df.index[:-(tahminEdilecekGunSayisi-1)], df[:-(tahminEdilecekGunSayisi-1)]['Close'], label='Train (Gerçek)', color='blue')
#plt.plot(df.index[-tahminEdilecekGunSayisi:], df[-tahminEdilecekGunSayisi:]['Close'], label='Test (Gerçek)', color='green')
plt.plot(df.index[:], df[:]['Close'], label='Actual Values', color='blue')


#plt.plot(tahmin_tarihleri, lstm_predictions, label='LSTM Tahminleri')
#plt.plot(tahmin_tarihleri, mlp_predictions, label='MLP Tahminleri')
plt.plot(tahmin_tarihleri, rnn_predictions, label='RNN Predictions', color='purple')
plt.plot(tahmin_tarihleri, cnn_predictions, label='CNN Predictions', color='red')

plt.legend(loc='lower left')
plt.show()

In [ ]:
# Performans ölçütlerini hesaplama
performance_metrics = []

true_values = df1[-tahminEdilecekGunSayisi:].values

for model_name, preds in zip(["LSTM", "RNN", "CNN", "MLP"], [lstm_predictions, rnn_predictions, cnn_predictions, mlp_predictions]):
 mse = mean_squared_error(true_values, preds)
 mae = mean_absolute_error(true_values, preds)
 #mape = np.mean(np.abs((true_values - preds) / true_values)) * 100
 mape = mean_absolute_percentage_error(true_values, preds)
 r2 = r2_score(true_values, preds)
 rmse = np.sqrt(mse)
 relative_rmse = rmse / np.mean(true_values)

 performance_metrics.append({
 "Model": model_name,
 "RMSE": rmse,
 "Relative RMSE": relative_rmse,
 "MAE": mae,
 "MAPE": mape,
 "R2": r2
 })

# Performans ölçütlerini DataFrame'e çevirme
performance_df = pd.DataFrame(performance_metrics)

# "Model" sütununun yazı tipini sola dayalı yapma
performance_df = performance_df.rename(columns={'Model': ' Model'})

# Performans ölçütlerini RMSE'ye göre sıralama (düşükten yükseğe)
sorted_performance_df = performance_df.sort_values(by='RMSE')

# Sıralı performans ölçütlerini yazdırma
sorted_performance_df_formatted = sorted_performance_df.to_string(index=False, formatters={'Model': '{:<5}'.format})
print(sorted_performance_df_formatted)


In [ ]:
# Veriyi Excel dosyasına kaydedin
#excel_file_name = f'/content/drive/MyDrive/Colab Notebooks/tez/DerinOgrenme_BorsaSonuclari_{start_date}_{end_date}_tahminEdilecekGunSayisi_{tahminEdilecekGunSayisi}.xlsx'
#sorted_performance_df.to_excel(excel_file_name, index=False)

In [ ]:
# =============================================================================
# TÜM PERİYOTLAR İÇİN TEST (1 gün, 10 gün, 21 gün)
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import os

# Sonuç dizini
SONUC_DIZINI = '/content/drive/MyDrive/Colab Notebooks/Sonuclar/1_DeepLearning'
os.makedirs(SONUC_DIZINI, exist_ok=True)

# Notebook bilgileri
NOTEBOOK_ADI = "1_DeepLearning"
VERI_TIPI = "az_veri"
KATEGORI = "Deep Learning"

print("=" * 80)
print(f" {NOTEBOOK_ADI} - TÜM PERİYOTLAR İÇİN TEST")
print(f" Veri Tipi: {VERI_TIPI.upper()}")
print("=" * 80)

# Tüm sonuçları toplayacak liste
tum_sonuclar = []

# Her periyot için test yap
for periyot_adi, gun_sayisi in TAHMIN_PERIYOTLARI.items():
    print(f"\n{'='*60}")
    print(f" {periyot_adi} ({gun_sayisi} iş günü) test ediliyor...")
    print('='*60)

    # Bu periyot için tahminleri al
    tahmin_gun = gun_sayisi

    # Gerçek değerler
    true_vals = df1[-tahmin_gun:].values.flatten()

    # Her model için tahmin ve metrik hesapla
    for model_name, model, is_mlp in [("LSTM", model_lstm, False),
                                        ("RNN", model_rnn, False),
                                        ("CNN", model_cnn, False),
                                        ("MLP", model_mlp, True)]:
        try:
            # Tahmin yap
            X, y = create_dataset(scaled_df1, time_step)
            if is_mlp:
                preds = model.predict(X[-tahmin_gun:, :])
            else:
                X_3D = X.reshape(X.shape[0], X.shape[1], 1)
                preds = model.predict(X_3D[-tahmin_gun:, :], verbose=0)

            # Ters ölçekleme
            preds = scaler.inverse_transform(preds.reshape(-1, 1)).flatten()

            # Boyut kontrolü
            min_len = min(len(true_vals), len(preds))
            true_v = true_vals[:min_len]
            pred_v = preds[:min_len]

            # Metrikler
            rmse = np.sqrt(mean_squared_error(true_v, pred_v))
            mae = mean_absolute_error(true_v, pred_v)
            r2 = r2_score(true_v, pred_v)
            mape = mean_absolute_percentage_error(true_v, pred_v) * 100

            tum_sonuclar.append({
                'Model': model_name,
                'Kategori': KATEGORI,
                'Veri_Tipi': VERI_TIPI,
                'Periyot': periyot_adi,
                'Periyot_Gun': gun_sayisi,
                'RMSE': round(rmse, 4),
                'MAE': round(mae, 4),
                'R2': round(r2, 4),
                'MAPE': round(mape, 2)
            })

            print(f"   {model_name}: RMSE={rmse:.4f}, R²={r2:.4f}")

        except Exception as e:
            print(f"   {model_name}: Hata - {e}")
            tum_sonuclar.append({
                'Model': model_name,
                'Kategori': KATEGORI,
                'Veri_Tipi': VERI_TIPI,
                'Periyot': periyot_adi,
                'Periyot_Gun': gun_sayisi,
                'RMSE': np.nan,
                'MAE': np.nan,
                'R2': np.nan,
                'MAPE': np.nan
            })

# DataFrame oluştur
sonuc_df = pd.DataFrame(tum_sonuclar)

print("\n" + "=" * 80)
print(" TÜM SONUÇLAR")
print("=" * 80)


In [ ]:
# =============================================================================
# SONUÇ TABLOSU - PERİYOTLARA GÖRE
# =============================================================================

# Pivot tablo oluştur (Model × Periyot)
print("\n" + "=" * 80)
print(" SONUÇ TABLOSU - RMSE")
print("=" * 80)

pivot_rmse = sonuc_df.pivot(index='Model', columns='Periyot', values='RMSE')
pivot_rmse = pivot_rmse[['1_gun', '10_gun', '21_gun']]  # Sıralama
print(pivot_rmse.to_string())

print("\n" + "=" * 80)
print(" SONUÇ TABLOSU - R²")
print("=" * 80)

pivot_r2 = sonuc_df.pivot(index='Model', columns='Periyot', values='R2')
pivot_r2 = pivot_r2[['1_gun', '10_gun', '21_gun']]
print(pivot_r2.to_string())

print("\n" + "=" * 80)
print(" SONUÇ TABLOSU - MAE")
print("=" * 80)

pivot_mae = sonuc_df.pivot(index='Model', columns='Periyot', values='MAE')
pivot_mae = pivot_mae[['1_gun', '10_gun', '21_gun']]
print(pivot_mae.to_string())

print("\n" + "=" * 80)
print(" SONUÇ TABLOSU - MAPE (%)")
print("=" * 80)

pivot_mape = sonuc_df.pivot(index='Model', columns='Periyot', values='MAPE')
pivot_mape = pivot_mape[['1_gun', '10_gun', '21_gun']]
print(pivot_mape.to_string())

# En iyi modeller
print("\n" + "=" * 80)
print(" EN İYİ MODELLER (RMSE bazında)")
print("=" * 80)
for periyot in ['1_gun', '10_gun', '21_gun']:
    subset = sonuc_df[sonuc_df['Periyot'] == periyot]
    if len(subset) > 0:
        best = subset.loc[subset['RMSE'].idxmin()]
        print(f"  {periyot:8} → {best['Model']:6} (RMSE: {best['RMSE']:.4f}, R²: {best['R2']:.4f})")


In [ ]:
# =============================================================================
# KARŞILAŞTIRMA GRAFİKLERİ
# =============================================================================
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'{NOTEBOOK_ADI} - Periyot Karşılaştırması ({VERI_TIPI})', fontsize=14, fontweight='bold')

models = ['LSTM', 'RNN', 'CNN', 'MLP']
periyotlar = ['1_gun', '10_gun', '21_gun']
x = np.arange(len(models))
width = 0.25
colors = ['#e74c3c', '#f39c12', '#27ae60']  # 1 gün, 10 gün, 21 gün

# 1. RMSE Karşılaştırması
ax1 = axes[0, 0]
for i, periyot in enumerate(periyotlar):
    values = [pivot_rmse.loc[m, periyot] if m in pivot_rmse.index else 0 for m in models]
    bars = ax1.bar(x + i*width, values, width, label=periyot, color=colors[i])
ax1.set_title('RMSE Karşılaştırması', fontweight='bold')
ax1.set_ylabel('RMSE')
ax1.set_xticks(x + width)
ax1.set_xticklabels(models)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. MAE Karşılaştırması
ax2 = axes[0, 1]
for i, periyot in enumerate(periyotlar):
    values = [pivot_mae.loc[m, periyot] if m in pivot_mae.index else 0 for m in models]
    bars = ax2.bar(x + i*width, values, width, label=periyot, color=colors[i])
ax2.set_title('MAE Karşılaştırması', fontweight='bold')
ax2.set_ylabel('MAE')
ax2.set_xticks(x + width)
ax2.set_xticklabels(models)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. R² Karşılaştırması
ax3 = axes[1, 0]
for i, periyot in enumerate(periyotlar):
    values = [pivot_r2.loc[m, periyot] if m in pivot_r2.index else 0 for m in models]
    bars = ax3.bar(x + i*width, values, width, label=periyot, color=colors[i])
ax3.set_title('R² Karşılaştırması', fontweight='bold')
ax3.set_ylabel('R²')
ax3.set_xticks(x + width)
ax3.set_xticklabels(models)
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. MAPE Karşılaştırması
ax4 = axes[1, 1]
for i, periyot in enumerate(periyotlar):
    values = [pivot_mape.loc[m, periyot] if m in pivot_mape.index else 0 for m in models]
    bars = ax4.bar(x + i*width, values, width, label=periyot, color=colors[i])
ax4.set_title('MAPE (%) Karşılaştırması', fontweight='bold')
ax4.set_ylabel('MAPE (%)')
ax4.set_xticks(x + width)
ax4.set_xticklabels(models)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f"{SONUC_DIZINI}/{NOTEBOOK_ADI}_karsilastirma.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n Grafik kaydedildi: {SONUC_DIZINI}/{NOTEBOOK_ADI}_karsilastirma.png")


In [ ]:
# =============================================================================
# EXCEL'E KAYDET
# =============================================================================

# Tüm sonuçları tek Excel dosyasına kaydet
excel_dosya = f"{SONUC_DIZINI}/{NOTEBOOK_ADI}_tum_sonuclar.xlsx"

with pd.ExcelWriter(excel_dosya, engine='openpyxl') as writer:
    # Ham veriler
    sonuc_df.to_excel(writer, sheet_name='Tum_Sonuclar', index=False)

    # Pivot tablolar
    pivot_rmse.to_excel(writer, sheet_name='RMSE')
    pivot_mae.to_excel(writer, sheet_name='MAE')
    pivot_r2.to_excel(writer, sheet_name='R2')
    pivot_mape.to_excel(writer, sheet_name='MAPE')

print(f"\n Tüm sonuçlar kaydedildi: {excel_dosya}")

# =============================================================================
# ÖZET
# =============================================================================
print("\n" + "=" * 80)
print(f" ÖZET - {NOTEBOOK_ADI}")
print("=" * 80)
print(f" Veri Tipi: {VERI_TIPI}")
print(f" Kategori: {KATEGORI}")
print(f" Test Edilen Modeller: {len(models)}")
print(f" Test Edilen Periyotlar: 1_gun, 10_gun, 21_gun")
print(f" Toplam Test: {len(sonuc_df)}")
print("=" * 80)
print(f"\n Kaydedilen dosyalar:")
print(f"   - {excel_dosya}")
print(f"   - {SONUC_DIZINI}/{NOTEBOOK_ADI}_karsilastirma.png")
